# Mandatory Assigment 2: Emotions and Digits
### By Andreas Haavik, Mikkel Rolf Moesgaard Pedersen, Clara Mille Kalø & Clara Truelsen
Student numbers: S145637, S146212, S143812, S144995

In [1]:
pip install  pandas plotly scikit-learn pytorch lightning torchvision torchmetrics tensorboard


  Preparing metadata (setup.py) ... done
  Obtaining dependency information for lightning from https://files.pythonhosted.org/packages/a0/4a/b7d4f62449d940ce43d4657322a14f5718815b648f9d2b0b23a195acb646/lightning-2.2.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 763.5 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/a8/95/affe174621ebc9b32544a918ae6cb758ac06061d0ee51d783e06906ceba1/torchvision-0.17.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for torchmetrics from https://files.pythonhosted.org/packages/cd/23/4bb4c1b78b57682a1309974a29bfdcbfa6fcf5476e698a4f0f22affa3799/torchmetrics-1.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for tensorboard from https://files.pythonhosted.org/packages/3a/d0/b97889ffa769e2d1fdebb632084d5e8b53fc299d43a537acee7ec0c021a3/tensorboard-2.16.2-py3-none-any.whl.metadata
  Obtaining dependency i

In [2]:
!pip install  pandas plotly scikit-learn torch lightning torchvision torchmetrics tensorboard

  Obtaining dependency information for lightning from https://files.pythonhosted.org/packages/a0/4a/b7d4f62449d940ce43d4657322a14f5718815b648f9d2b0b23a195acb646/lightning-2.2.1-py3-none-any.whl.metadata
  Using cached lightning-2.2.1-py3-none-any.whl.metadata (56 kB)
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/a8/95/affe174621ebc9b32544a918ae6cb758ac06061d0ee51d783e06906ceba1/torchvision-0.17.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached torchvision-0.17.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Obtaining dependency information for torchmetrics from https://files.pythonhosted.org/packages/cd/23/4bb4c1b78b57682a1309974a29bfdcbfa6fcf5476e698a4f0f22affa3799/torchmetrics-1.3.1-py3-none-any.whl.metadata
  Using cached torchmetrics-1.3.1-py3-none-any.whl.metadata (19 kB)
  Obtaining dependency information for tensorboard from https://files.pythonhosted.org/packages/3a/d0/b97889ffa769e2d1fdebb632084d5e8b53fc299d43

# **Task 1: Emotion Classification**



### 1.1 Importing relevant libraries


In [3]:
from typing import Callable # Extra: read about type hints: https://docs.python.org/3/library/typing.html
import warnings


# needs to run before importing pandas. Silences a harmless pandas FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import plotly.express as px # for interactive plotting


# disable in-cell scientific notation
pd.options.display.float_format = '{:.0f}'.format

# set plotly as the plotting backend for pandas
pd.options.plotting.backend = "plotly"

### 1.2 Create Loader Class
The CSV Loader class enables us to load CSV data and transforms the data into a pd dataframe.


In [4]:
class CSVLoader:
    """ Load a CSV file into a pandas DataFrame."""

    @staticmethod
    def fetch(path : str, **kwargs) -> pd.DataFrame:
        return pd.read_csv(path, **kwargs)

    def load(self, path : str, prep_func : Callable | None, **kwargs) -> pd.DataFrame:
        data = self.fetch(path, **kwargs)
        if isinstance(prep_func, Callable):
            return prep_func(data)
        return data

### 1.3 Import Data
Data uploaded via Files and stored in Google Drive.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
def preprocess_emotions(df: pd.DataFrame) -> pd.DataFrame:
    """ A data preparation function for the emotions dataset."""
    return (
        df
        .drop(columns=['Unnamed: 0'], errors='ignore')
        .astype({"text": 'string', "emotion": 'category'})
    )

dataf = CSVLoader().load(
    path='/content/drive/MyDrive/UJeng.csv',
    prep_func=preprocess_emotions
)

In [ ]:
dataf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282313 entries, 0 to 282312
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype   
---  ------   --------------   -----   
 0   text     282313 non-null  string  
 1   emotion  282313 non-null  category
dtypes: category(1), string(1)
memory usage: 2.4 MB


# 1.4 EDA
This section explores the data through detailed analysis.

Questions to answer:

**1.4.1:** What does each feature in the dataset represent?

**1.4.2:** What is the distribution of emotions in the dataset?

**1.4.3:** Are there any missing values in the dataset? if yes, how many and in which feature?

**1.4.4:** Explore interesting characteristics of the text, such as lenght and complexity (text token ratio)

**1.4.5:** Are there any features which are highly correlated (above 0.7)? And how can we visualize it? and what features are positively or negatively correlated with specific emotions?

The above questions are relevant to explore to understand and potentially alter the dataset to obtain the best results, allowing for exploration of how English Facebook posts can be classified into the five emotions.


### 1.4.1: Dataset exploration: what does each feature in the dataset represent?

In [ ]:
print('First of we print top row, showing which attributes are included in the dataset and allows us to explore the top 5 features')
print(dataf.head())


print('Next, we explore all datatypes present in the dataset')
print(dataf.dtypes)

print('Lastly, we look into how many rows and attributes the dataset contains')
dataf.shape



First of we print top row, showing which attributes are included in the dataset and allows us to explore the top 5 features
                                                text       emotion
0  [PERSON] , is one of my favorite people. [PHOT...           joy
1  [PHOTO] STARTED FROM THE BOTTOM NOW I'M HERE !...  anticipation
2  UNWRAPPED TOY is all you need to get in! OPEN ...  anticipation
3  If anyone's in Coburg tonight, my old friend [...  anticipation
4  So super proud of my mom [PERSON] I could scre...           joy
Next, we explore all datatypes present in the dataset
text         string
emotion    category
dtype: object
Lastly, we look into how many rows and attributes the dataset contains


(282313, 2)

**The dataset contains 282312 rows and 2 attributes: text (datatype = string) and emotion (datatype = category).**



### 1.4.2: Distribution of emotion in the dataset

In [ ]:
"""Importing necessary libraries"""
import plotly.express as px

"""Creating a bar chart, to show frequency of emotion in the dataset"""

figBar = px.bar(dataf['emotion'].value_counts().reset_index(),
             x='index',
             y='emotion',
             labels={'index': 'Emotion', 'emotion': 'Count'},
             title='Distribution of Emotions - Bar Chart')
figBar.show()


**This bar chart clearly shows that a large amount of the emotion in the facebook posts are either 'anticipation' and 'joy'**

To further depict how the different emotion relate to the whole, we have created a Pie Chart. The Pie Chart includes a visualization of how many percentage each emotion makes up of the entire dataset.

In [ ]:
figPie = px.pie(dataf, names='emotion', title='Distribution of Emotions - Pie Chart')
figPie.show()


### 1.4.3: Missing values in dataset

In [ ]:
import numpy as np

# Categorizing missing values
def categorize_missing_values(dataf):
    missing_data = dataf.isnull()
    missing_columns = missing_data.any()

    summary = {
        'total_missing': missing_data.sum().sum(),
        'missing_per_column': missing_data.sum(),
        'percent_missing_per_column': missing_data.mean() * 100
    }

    print("Overall Missing Data:", summary['total_missing'])
    print("\nMissing Data Per Column:\n", summary['missing_per_column'])
    print("\nPercentage of Missing Data Per Column:\n", summary['percent_missing_per_column'])

    # Identifying columns with any missing values
    cols_with_missing = missing_columns[missing_columns].index.tolist()
    print("\nColumns with missing values:", cols_with_missing)

categorize_missing_values(dataf)



Overall Missing Data: 0

Missing Data Per Column:
 text       0
emotion    0
dtype: int64

Percentage of Missing Data Per Column:
 text      0
emotion   0
dtype: float64

Columns with missing values: []


**As seen from the code above, the dataset is complete and is not missing any values**

### 1.4.4 Analysing the characteristics of the text - type-token ratio

In [ ]:
from collections import Counter
import nltk
nltk.download('punkt')  # Download the necessary resources for tokenization

# Function to calculate type-token ratio
def calculate_ttr(text):
    tokens = nltk.word_tokenize(text.lower())  # Tokenize the text and convert to lowercase
    num_tokens = len(tokens)
    num_types = len(set(tokens))
    if num_tokens > 0:
        ttr = num_types / num_tokens
    else:
        ttr = 0
    return ttr

# Assuming the column containing text is named 'text_column'
texts = dataf['text']

# Calculate TTR for each text entry
ttrs = []
for text in texts:
    ttr = calculate_ttr(str(text))  # Convert to string to handle potential NaN values
    ttrs.append(ttr)

# Print the calculated TTRs
for i, ttr in enumerate(ttrs):
    print(f'Text {i+1}: Type-Token Ratio = {ttr:.4f}')

# Calculate average TTR
average_ttr = sum(ttrs) / len(ttrs)
print(f'Average Type-Token Ratio = {average_ttr:.4f}')

# Creating a new column containing all the TTR values
dataf['TTR']=ttrs
print(dataf)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Streaming output truncated to the last 5000 lines.
Text 69443: Type-Token Ratio = 0.7073
Text 69444: Type-Token Ratio = 0.7000
Text 69445: Type-Token Ratio = 0.9600
Text 69446: Type-Token Ratio = 0.6667
Text 69447: Type-Token Ratio = 0.7778
Text 69448: Type-Token Ratio = 0.5385
Text 69449: Type-Token Ratio = 0.8788
Text 69450: Type-Token Ratio = 0.7143
Text 69451: Type-Token Ratio = 1.0000
Text 69452: Type-Token Ratio = 1.0000
Text 69453: Type-Token Ratio = 0.5556
Text 69454: Type-Token Ratio = 0.8095
Text 69455: Type-Token Ratio = 0.8125
Text 69456: Type-Token Ratio = 1.0000
Text 69457: Type-Token Ratio = 0.7333
Text 69458: Type-Token Ratio = 0.8852
Text 69459: Type-Token Ratio = 0.6393
Text 69460: Type-Token Ratio = 1.0000
Text 69461: Type-Token Ratio = 1.0000
Text 69462: Type-Token Ratio = 0.7674
Text 69463: Type-Token Ratio = 0.8636
Text 69464: Type-Token Ratio = 0.8000
Text 69465: Type-Token Ratio = 0.8158
Text 69466: Type-Token Ratio = 0.7826
Text 69467: Type-Token Ratio = 0.6667

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
Text 277329: Type-Token Ratio = 0.8857
Text 277330: Type-Token Ratio = 0.7619
Text 277331: Type-Token Ratio = 0.6667
Text 277332: Type-Token Ratio = 0.8966
Text 277333: Type-Token Ratio = 0.7895
Text 277334: Type-Token Ratio = 0.7879
Text 277335: Type-Token Ratio = 0.9231
Text 277336: Type-Token Ratio = 0.8000
Text 277337: Type-Token Ratio = 0.7143
Text 277338: Type-Token Ratio = 0.6744
Text 277339: Type-Token Ratio = 1.0000
Text 277340: Type-Token Ratio = 0.8000
Text 277341: Type-Token Ratio = 0.8571
Text 277342: Type-Token Ratio = 0.7164
Text 277343: Type-Token Ratio = 0.8750
Text 277344: Type-Token Ratio = 0.9167
Text 277345: Type-Token Ratio = 0.8400
Text 277346: Type-Token Ratio = 0.7895
Text 277347: Type-Token Ratio = 1.0000
Text 277348: Type-Token Ratio = 1.0000
Text 277349: Type-Token Ratio = 1.0000
Text 277350: Type-Token Ratio = 0.7317
Text 277351: Type-Token Ratio = 0.6230
Text 277352: Type-Token Ratio = 0.8684
Text 277353: 

As seen from the results above, where the average type-token ration in all text fields is 0.8, it is evident that the text fields consists of mostly unique words, but some still use the same words from time to time.

The below code will give a more in depth insight into the distribution of each text fields type-token ratio.

In [ ]:
# Below we assign the text column to a new variable
text_ttr = dataf['TTR']

# Initialize counters
count_above_1 = 0
count_below_1 = 0

# Iterate through the values
for value in text_ttr:
    if value == 1:  # Compare each value to 1
        count_above_1 += 1
    elif value < 1:  # Compare each value to 1
        count_below_1 += 1

# Print the results
print(f'Total number of text fields with a TTR of 1: {count_above_1}')
print(f'Total number of TTR below 1: {count_below_1}')



Total number of text fields with a TTR of 1: 38374
Total number of TTR below 1: 243939


**In conclusion, 38374 of the text field in the dataset has a TTR of 1, meaning that these only contain unique words, whereas 243939 text fields have a TTR below 1, thus containing some of the same words.**

### **Before answering the last question in the EDA, some data cleaning is necessary to be able to look closely at the correlation between emotions and text.**
Question 1.4.5 will be answered in the modelling section

# 1.5 Data Cleaning

Cleaning the text so that "stopwords"/ standard words like "the" and so on are removed. Furthermore, everything is turned into lowercase and special signs are remowed. Lemmetization standardizes word forms to their root/base form to reduce the vocabulary size and improve model training efficiency.

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove special characters and numbers
    text = re.sub(r'\W+|\d+', ' ', text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    text = ' '.join(filtered_tokens)

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    text = ' '.join(lemmatized_tokens)
    return text

# Applying the cleaning function to each entry in the column
dataf['cleaned_text'] = dataf['text'].apply(clean_text)

cleaned_dataf = dataf[['cleaned_text', 'emotion']]

print(dataf['cleaned_text'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0         person one favorite people photo fan amy bumpe...
1         photo started bottom next saturday new residen...
2             unwrapped toy need get open public wanna miss
3         anyone coburg tonight old friend person playin...
4         super proud mom person could scream busy made ...
                                ...                        
282308         photo interview panda excited new begin year
282309    photo new ink got today done extremely talente...
282310                    photo ce sooooooir unchained mind
282311                                    photo friday rome
282312    silent prayer march st xavier church vile parl...
Name: cleaned_text, Length: 282313, dtype: object


Below we look at the newly created dataframe to ensure that the data has been cleaned

In [ ]:
cleaned_dataf.head(2)

,cleaned_text,emotion
0,person one favorite people photo fan amy bumpe...,joy
1,photo started bottom next saturday new residen...,anticipation


Now we vectorize the "text" => convert text data into numerical form using TF-IDF - Term Frequency-Inverse Document Frequency. We also split the data into train and test subsets so the models can be tested on unseen data.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(cleaned_dataf['cleaned_text'], cleaned_dataf['emotion'], test_size=0.2, random_state=42)


# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [ ]:
print(X_train_tfidf)

  (0, 91658)	0.2661699847003645
  (0, 92278)	0.08146551471779767
  (0, 134312)	0.5758104459663071
  (0, 90018)	0.5508279035558535
  (0, 38171)	0.5362348926885661
  (1, 4682)	0.37974857164267234
  (1, 113020)	0.34088760997467676
  (1, 27462)	0.3457493590590517
  (1, 16829)	0.29168567821974256
  (1, 6856)	0.5999550480841574
  (1, 131717)	0.23988103089456325
  (1, 136176)	0.22571033326279008
  (1, 68186)	0.2253412755774559
  (1, 91658)	0.09244363145784884
  (1, 92278)	0.08488148685405553
  (2, 120703)	0.11985215250206428
  (2, 43542)	0.14790900274826532
  (2, 37788)	0.1541153880658209
  (2, 93153)	0.1551695079954244
  (2, 38340)	0.2791528912367328
  (2, 46137)	0.2806735395497185
  (2, 102130)	0.149000671431265
  (2, 6691)	0.5734629056383987
  (2, 70369)	0.10502216465110141
  (2, 87308)	0.18034477531337695
  :	:
  (225847, 25960)	0.35773375040204963
  (225847, 66374)	0.2883951454389087
  (225847, 46866)	0.2906505460177461
  (225847, 36705)	0.26154322089044096
  (225847, 115097)	0.226851231

# 1.6 Modelling
This section will look further into the dataset, exploring the correlations between the text's complexity/length or other characteristics with the emotions. This will be done through different relevant model building. Through relevant models this section will explore question 1.4.5 and other relevant data analysis aspects.

In this section, we build three different classification models: Logistic Regression, Naive Bayes, and Random Forrest

### Logistic Regression


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Initialize Logistic Regression model
logreg = LogisticRegression(max_iter=1000)

# Fit the model on training data
logreg.fit(X_train_tfidf, y_train)

# Predict the emotions on the test set
y_pred = logreg.predict(X_test_tfidf)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.6313160830986664

Classification Report:
               precision    recall  f1-score   support

       anger       0.67      0.40      0.50      3379
anticipation       0.65      0.68      0.66     23031
        fear       0.73      0.16      0.26       603
         joy       0.60      0.65      0.62     22119
     sadness       0.67      0.56      0.61      7331

    accuracy                           0.63     56463
   macro avg       0.66      0.49      0.53     56463
weighted avg       0.63      0.63      0.63     56463

Confusion Matrix:
 [[ 1347   629     3  1009   391]
 [  141 15651     7  6614   618]
 [   32   148    95   135   193]
 [  207  6650     1 14476   785]
 [  274   975    24  1981  4077]]


**conclusion:**

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Initialize the Naive Bayes model
naive_bayes_classifier = MultinomialNB()

# Fit the model on your training data
naive_bayes_classifier.fit(X_train_tfidf, y_train)

# Predict the emotions on the test set
y_pred_nb = naive_bayes_classifier.predict(X_test_tfidf)

# Evaluate the model
print("Naive Bayes Model Accuracy:", accuracy_score(y_test, y_pred_nb))
print("\nNaive Bayes Classification Report:\n", classification_report(y_test, y_pred_nb))
print("Naive Bayes Confusion Matrix:\n", confusion_matrix(y_test, y_pred_nb))

Naive Bayes Model Accuracy: 0.5797956183695517


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.




Naive Bayes Classification Report:
               precision    recall  f1-score   support

       anger       0.96      0.08      0.15      3379
anticipation       0.59      0.75      0.66     23031
        fear       0.00      0.00      0.00       603
         joy       0.55      0.62      0.58     22119
     sadness       0.82      0.22      0.35      7331

    accuracy                           0.58     56463
   macro avg       0.58      0.33      0.35     56463
weighted avg       0.62      0.58      0.55     56463

Naive Bayes Confusion Matrix:
 [[  277  1293     0  1672   137]
 [    0 17170     0  5795    66]
 [    2   333     0   204    64]
 [    6  8363     0 13651    99]
 [    5  2052     0  3635  1639]]


**Conclusion:**

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Initialize the Random Forest classifier
# n_estimators is the number of trees in the forest
# random_state is used for reproducibility of the results
random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model on your training data
random_forest_classifier.fit(X_train_tfidf, y_train)

# Predict the emotions on the test set
y_pred_rf = random_forest_classifier.predict(X_test_tfidf)

# Evaluate the model
print("Random Forest Model Accuracy:", accuracy_score(y_test, y_pred_rf))
print("\nRandom Forest Classification Report:\n", classification_report(y_test, y_pred_rf))
print("Random Forest Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))